<a href="https://colab.research.google.com/github/tpothineni/MLProject/blob/main/Example_HateExplain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tpothineni/MLProject.git

Cloning into 'MLProject'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 101 (delta 29), reused 89 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (101/101), 2.34 MiB | 6.17 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
cd MLProject/

/content/MLProject


In [4]:
!mkdir Saved/
!mkdir explanations_dicts/

In [5]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip  -P Data/

--2024-11-09 17:36:34--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2024-11-09 17:36:34--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2024-11-09 17:36:35--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [6]:
!unzip Data/glove.42B.300d.zip -d Data/

Archive:  Data/glove.42B.300d.zip
  inflating: Data/glove.42B.300d.txt  


In [7]:
!rm Data/glove.42B.300d.zip

In [18]:
!pip install -r requirements.txt

  Using cached neptune-client-0.4.107.tar.gz (86 kB)
  Preparing metadata (setup.py) ... done
  Using cached knockknock-0.1.7-py3-none-any.whl.metadata (11 kB)
  Using cached numpy-1.16.3.zip (5.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.43.0-py2.py3-none-any.whl.metadata (49 kB)
  Using cached Keras-2.3.1-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached waiting-1.4.1.tar.gz (7.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached ekphrasis-0.5.1.tar.gz (80 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-2.5.1-py3-none-any.whl.metadata (42 kB)
  Using cached lime-0.2.0.1.tar.gz (275 kB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.2.1.tar.gz (40.3 MB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.0 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies d

In [9]:
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec('Data/glove.42B.300d.txt', 'Data/glove.42B.300d_w2v.txt')
word2vecmodel1 = KeyedVectors.load_word2vec_format('Data/glove.42B.300d_w2v.txt', binary=False)
word2vecmodel1.save("Data/word2vec.model")

In [10]:
import gc
del word2vecmodel1
gc.collect()

0

In [11]:
!rm Data/glove.42B.300d.txt
!rm Data/glove.42B.300d_w2v.txt

In [12]:
from manual_training_inference import *

ModuleNotFoundError: No module named 'neptune'

In [ ]:
path_file='best_model_json/bestModel_birnnscrat.json'
with open(path_file,mode='r') as f:
    params = json.load(f)
for key in params:
    if params[key] == 'True':
          params[key]=True
    elif params[key] == 'False':
          params[key]=False
    if( key in ['batch_size','num_classes','hidden_size','supervised_layer_pos','num_supervised_heads','random_seed','max_length']):
        if(params[key]!='N/A'):
            params[key]=int(params[key])

    if((key == 'weights') and (params['auto_weights']==False)):
        params[key] = ast.literal_eval(params[key])

##### change in logging to output the results to neptune
params['logging']='local'
params['device']='cpu'
params['best_params']=False

if torch.cuda.is_available() and params['device']=='cuda':
    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")
else:
    print('Since you dont want to use GPU, using the CPU instead.')
    device = torch.device("cpu")


#### Few handy keys that you can directly change.
params['variance']=1
params['epochs']=5
params['to_save']=True
params['num_classes']=2
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']
if(params['num_classes']==2 and (params['auto_weights']==False)):
      params['weights']=[1.0,1.0]

#for att_lambda in [0.001,0.01,0.1,1,10,100]
train_model(params,device)

In [ ]:
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']
if(params['num_classes']==2 and (params['auto_weights']==False)):
      params['weights']=[1.0,1.0]

#for att_lambda in [0.001,0.01,0.1,1,10,100]
train_model(params,device)

In [ ]:
import gc
gc.collect()

In [ ]:
!python testing_with_rational.py birnn_scrat 100
!python testing_for_bias.py birnn_scrat 100

In [ ]:
!ls explanations_dicts



---

---




**Bias Calculation**

---



In [ ]:
from collections import Counter,defaultdict
from tqdm.notebook import tqdm
import json
import numpy as np

In [ ]:
# get_annotated_data method is used to load the dataset
from Preprocess.dataCollect import get_annotated_data

In [ ]:
dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

params = {}

# We need to load the dataset with the labels as 'toxic' and 'non-toxic'.
# We consider hatespeech and offensive as toxic and normal as non-toxic.
params['num_classes']=2
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [ ]:
data_all_labelled

In [ ]:
def generate_target_information(dataset):
    final_target_output = defaultdict(list)
    all_communities_selected = []

    for each in dataset.iterrows():
        # All the target communities tagged for this post
        all_targets = each[1]['target1']+each[1]['target2']+each[1]['target3']
        community_dict = dict(Counter(all_targets))

        # Select only those communities which are present more than once.
        for key in community_dict:
            if community_dict[key]>1:
                final_target_output[each[1]['post_id']].append(key)
                all_communities_selected.append(key)

        # If no community is selected based on majority voting then we don't select any community
        if each[1]['post_id'] not in final_target_output:
            final_target_output[each[1]['post_id']].append('None')
            all_communities_selected.append(key)

    return final_target_output, all_communities_selected

In [ ]:
target_information, all_communities_selected = generate_target_information(data_all_labelled)

In [ ]:
community_count_dict = Counter(all_communities_selected)

# We remove None and Other from dictionary
community_count_dict.pop('None')
community_count_dict.pop('Other')

# For the bias calculation, we are considering the top 10 communites based on their count
list_selected_community = [community for community, value in community_count_dict.most_common(10)]
list_selected_community

In [ ]:
# Based on the top 10 communities, we filter the target_information
# This will remove the other communities from the calculation

final_target_information ={}
for each in target_information:
    temp = list(set(target_information[each])&set(list_selected_community))
    if len(temp) == 0:
        final_target_information[each] = None
    else:
        final_target_information[each] = temp

In [ ]:
# Add a new column 'final_target_category' which will contain the selected target community names
data_all_labelled['final_target_category'] = data_all_labelled['post_id'].map(final_target_information)

In [ ]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
postpost_id_divisions_path = './Data/post_id_divisions.json'

with open(postpost_id_divisions_path, 'r') as fp:
    post_id_dict=json.load(fp)

data_all_labelled_bias = data_all_labelled[data_all_labelled['post_id'].isin(post_id_dict['test'])]
data_all_labelled_bias

In [ ]:
from sklearn.metrics import roc_auc_score

# The names of the files which will contain the bias output for each model
bias_score_file_mapping={
    'BiRNN-Attn':'bestModel_birnnscrat_bias.json',
}

# The parent folder path of the bias output files
parent_path = './explanations_dicts/'

# The bias methods that will be considered
method_list = ['subgroup', 'bpsn', 'bnsp']

community_list = list(list_selected_community)

In [ ]:
# This function is used to convert the classification into a [0-1] score
# with a value of 0 meaning non-toxic and 1 meaning toxic
def convert_to_score(label_name, label_dict):
    if label_name=='non-toxic':
        return 1-label_dict[label_name]
    else:
        return label_dict[label_name]

In [ ]:
# Function to divide the ids into postive or class class based on the method.
def bias_evaluation_metric(dataset, method, community):
    positive_ids = []
    negative_ids = []
    if method=='subgroup':
        for eachrow in dataset.iterrows():
            if eachrow[1]['final_target_category'] == None:
                continue
            if community in eachrow[1]['final_target_category']:
                if eachrow[1]['final_label'] =='non-toxic':
                    negative_ids.append(eachrow[1]['post_id'])
                else:
                    positive_ids.append(eachrow[1]['post_id'])
            else:
                pass
    elif method=='bpsn':
        for eachrow in dataset.iterrows():
            if eachrow[1]['final_target_category'] == None:
                continue
            if community in eachrow[1]['final_target_category']:
#                 print(eachrow[1]['final_label'])
                if eachrow[1]['final_label'] =='non-toxic':
                    negative_ids.append(eachrow[1]['post_id'])
                else:
                    pass
            else:
                if eachrow[1]['final_label'] !='non-toxic':
                    positive_ids.append(eachrow[1]['post_id'])
                else:
                    pass
    elif method=='bnsp':
        for eachrow in dataset.iterrows():
            if eachrow[1]['final_target_category'] == None:
                continue
            if community in eachrow[1]['final_target_category']:
                if eachrow[1]['final_label'] !='non-toxic':
                    positive_ids.append(eachrow[1]['post_id'])
                else:
                    pass
            else:
                if eachrow[1]['final_label'] =='non-toxic':
                    negative_ids.append(eachrow[1]['post_id'])
                else:
                    pass
    else:
        print('Incorrect option selected!!!')

    return {'positiveID':positive_ids, 'negativeID':negative_ids}

In [ ]:
final_bias_dictionary = defaultdict(lambda: defaultdict(dict))

# We load each of the model bias output file and compute the bias score using each method for all the community
for each_model in tqdm(bias_score_file_mapping):
    total_data ={}
    with open(parent_path+bias_score_file_mapping[each_model]) as fp:
        for line in fp:
            data = json.loads(line)
            total_data[data['annotation_id']] = data
    for each_method in method_list:
        for each_community in community_list:
            community_data = bias_evaluation_metric(data_all_labelled_bias, each_method, each_community)
            truth_values = []
            prediction_values = []


            label_to_value = {'toxic':1.0, 'non-toxic':0.0}
            for each in community_data['positiveID']:
                truth_values.append(label_to_value[total_data[each]['ground_truth']])
                prediction_values.append(convert_to_score(total_data[each]['classification'], total_data[each]['classification_scores']))

            for each in community_data['negativeID']:
                truth_values.append(label_to_value[total_data[each]['ground_truth']])
                prediction_values.append(convert_to_score(total_data[each]['classification'], total_data[each]['classification_scores']))

            roc_output_value = roc_auc_score(truth_values, prediction_values)
            final_bias_dictionary[each_model][each_method][each_community] = roc_output_value

In [ ]:
%precision 4

In [ ]:
# To combine the per-identity Bias AUCs into one overall measure, we calculate their generalized mean as defined below:
power_value = -5
num_communities = len(community_list)

for each_model in final_bias_dictionary:
    for each_method in final_bias_dictionary[each_model]:
        temp_value =[]
        for each_community in final_bias_dictionary[each_model][each_method]:
            temp_value.append(pow(final_bias_dictionary[each_model][each_method][each_community], power_value))
        print(each_model, each_method, pow(np.sum(temp_value)/num_communities, 1/power_value))



---


**Calculate Explainability**

---



In [ ]:
import json
from tqdm.notebook import tqdm
import more_itertools as mit
import os

In [ ]:
# get_annotated_data method is used to load the dataset
from Preprocess import *
from Preprocess.dataCollect import *

In [ ]:
dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

# We need to load the dataset with the labels as 'hatespeech', 'offensive', and 'normal' (3-class).

params = {}
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [ ]:
# The important key here is the 'bert_token'. Set it to True for Bert based models and False for Others.

params_data={
    'include_special':False,  #True is want to include <url> in place of urls if False will be removed
    'bert_tokens':False, #True /False
    'type_attention':'softmax', #softmax
    'set_decay':0.1,
    'majority':2,
    'max_length':128,
    'variance':10,
    'window':4,
    'alpha':0.5,
    'p_value':0.8,
    'method':'additive',
    'decay':False,
    'normalized':False,
    'not_recollect':True,
}


if(params_data['bert_tokens']):
    print('Loading BERT tokenizer...')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
else:
    print('Loading Normal tokenizer...')
    tokenizer=None

In [ ]:
# Load the whole dataset and get the tokenwise rationales
def get_training_data(data):
    post_ids_list=[]
    text_list=[]
    attention_list=[]
    label_list=[]

    final_binny_output = []
    print('total_data',len(data))
    for index,row in tqdm(data.iterrows(),total=len(data)):
        annotation=row['final_label']

        text=row['text']
        post_id=row['post_id']
        annotation_list=[row['label1'],row['label2'],row['label3']]
        tokens_all = list(row['text'])
#         attention_masks =  [list(row['explain1']),list(row['explain2']),list(row['explain1'])]

        if(annotation!= 'undecided'):
            tokens_all,attention_masks=returnMask(row, params_data, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output

In [ ]:
training_data=get_training_data(data_all_labelled)

In [ ]:
# https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each==1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each)== int:
            start = each
            end = each+1
        elif len(each) == 2:
            start = each[0]
            end = each[1]+1
        else:
            print('error')

        output.append({"docid":post_id,
              "end_sentence": -1,
              "end_token": end,
              "start_sentence": -1,
              "start_token": start,
              "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output

# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):
    final_output = []

    if save_split:
        train_fp = open(save_path+'train.jsonl', 'w')
        val_fp = open(save_path+'val.jsonl', 'w')
        test_fp = open(save_path+'test.jsonl', 'w')

    for tcount, eachrow in enumerate(dataset):

        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]

        if majority_label=='normal':
            continue

        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))

        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]


        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)

        if save_split:
            if not os.path.exists(save_path+'docs'):
                os.makedirs(save_path+'docs')

            with open(save_path+'docs/'+post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))

            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp)+'\n')

            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp)+'\n')

            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp)+'\n')
            else:
                print(post_id)

    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()

    return final_output

In [ ]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('./Data/post_id_divisions.json') as fp:
    id_division = json.load(fp)

In [ ]:
!mkdir ./Data/Evaluation
!mkdir ./Data/Evaluation/Model_Eval

In [ ]:
method = 'union'
save_split = True
save_path = './Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
output_eraser = convert_to_eraser_format(training_data, method, save_split, save_path, id_division)

In [ ]:
!ls Data/Evaluation/Model_Eval/

In [ ]:
!ls

In [ ]:
cd eraserbenchmark/

In [ ]:
!ls

In [ ]:
!PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test  --data_dir ../Data/Evaluation/Model_Eval --results ../explanations_dicts/bestModel_birnnscrat_100_explanation_top5.json --score_file ../model_explain_output.json

In [ ]:
# print the required results
with open('../model_explain_output.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])